In [1]:
import folium
import pandas as pd

# Load the data from the CSV file
data = pd.read_csv('data/posiciones_ovitrampas2.csv')

# Create a map centered on the first data point
map = folium.Map(location=[data['lat'].iloc[0], data['long'].iloc[0]], zoom_start=14, control_scale=True)


# Add markers for each data point
for index, row in data.iterrows():
	fecha = row['fecha']  # Color verde por defecto si no hay mapeo
	folium.Marker([row['lat'], row['long']]).add_to(map)

# Display the map
map

In [2]:
import io
from PIL import Image

img_data = map._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('ovitrampas.png')

In [3]:
import pandas as pd
import folium
import json

# Load the data from the CSV file
data = pd.read_csv('data/posiciones_ovitrampas2.csv')

# Calcular las coordenadas mínimas y máximas de latitud y longitud
min_lat = data['lat'].min()
max_lat = data['lat'].max()
min_lon = data['long'].min()
max_lon = data['long'].max()

# Calcular el centro del ROI
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Calcular la diferencia entre las coordenadas máximas y mínimas
diff_lat = max_lat - min_lat
diff_lon = max_lon - min_lon

# Agregar un margen de 2 km a la diferencia
margin = 0.02  # 2 km en grados
diff_lat += margin
diff_lon += margin

# Calcular la mitad de la diferencia ajustada
half_diff_lat = diff_lat / 2
half_diff_lon = diff_lon / 2

if half_diff_lat > half_diff_lon:
	half_diff_lon = half_diff_lat
else:
	half_diff_lat = half_diff_lon

print(half_diff_lat == half_diff_lon)

# Calcular las coordenadas del ROI
roi_lat = [center_lat - half_diff_lat, center_lat + half_diff_lat]
roi_lon = [center_lon - half_diff_lon, center_lon + half_diff_lon]

# Crear un mapa centrado en el centro del ROI
map = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# Agregar un marcador para cada punto en el DataFrame
for index, row in data.iterrows():
	folium.Marker([row['lat'], row['long']]).add_to(map)

# Agregar un rectángulo que representa el ROI

print(f"{roi_lat, roi_lon}")

folium.Rectangle(bounds=[[roi_lat[0], roi_lon[0]], [roi_lat[1], roi_lon[1]]], color='red', fill=False).add_to(map)

# Mostrar el mapa
map

([-31.85280432, -31.810500800000003], [-60.539312540000005, -60.49700902])


In [5]:
import io
from PIL import Image

img_data = map._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('ovitrampasConROI.png')

In [ ]:
# Define the file path
file_path = "data/param.json"

# Read the JSON file
with open(file_path, "r") as json_file:
	data = json.load(json_file)

# Add a new key-value pair
data["roi"] = {
	"lon": roi_lon,
	"lat": roi_lat
}

# Save the updated JSON file
with open(file_path, "w") as json_file:
	json.dump(data, json_file, indent=2)